In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [4]:
import pickle
with open('../scraped.pkl', 'rb') as f:
    obj = pickle.load(f)
obj['texts']

['\nStart right now. The younger you are, the more your savings can grow.\n\n\n\nBy\n\nAmy Fontinelle\n\n\n\n\n\n\n\n\n \nFull Bio\n\n\n\n \n\n\nLinkedIn\n \n\n \n\n\nTwitter\n \n\n\n\nAmy Fontinelle has more than 15 years of experience covering personal finance, corporate finance and investing.\n\n\n\nLearn about our \neditorial policies\n\n\n\n\nUpdated April 11, 2023\n\n\n\n\n\n\nReviewed by\nAndy Smith\n\n\n\n\n\n\n\nFact checked by\n\nSuzanne Kvilhaug\n\n\n\n\n\n\n\n\n \n\n\n\n\nFact checked by\n\nSuzanne Kvilhaug\n\nFull Bio\n\nSuzanne is a content marketer, writer, and fact-checker.\xa0She holds a Bachelor of Science in Finance degree from Bridgewater State University and helps develop content strategies for financial brands.\n\n\n\nLearn about our \neditorial policies\n\n\n\n\n\n\n',
 '\nTaking the time to learn a few basic financial rules can help you build a healthy financial future.Start an emergency fund and pay yourself every month.Saving for retirement is an integral\xa0p

In [4]:

df = pd.read_csv('../dataset_google-search-scraper_2023-04-30_21-21-04-757.csv')

cols = []
for i in df.columns:
    if 'url' in i and 'organicResults' in i:
        cols.append(i)
    if 'searchQuery/term' in i: 
        cols.insert(0, i)
urls = df[cols]
urls = urls.melt(id_vars=['searchQuery/term'], var_name='result_num', value_name='url').dropna()
urls
header_length = []
for url in tqdm(urls['url']):
    try: 
        response = requests.get(url, timeout=10)
    except Exception as e:
        header_length.append(0)
        continue
    html_content = response.content

    # Use Beautiful Soup to parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all headers and the text underneath them
    headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    header_length.append(len(headers))
urls['header_length'] = header_length

  8%|▊         | 73/921 [01:12<08:40,  1.63it/s]

In [ ]:
df.to_csv('cleaned_urls.csv', index=False)

In [ ]:
headers = []
texts = []
issues = []
for url in tqdm(urls['url'][1:]):
    # Get the HTML content from the URL
    try: 
        response = requests.get(url, timeout=10)
    except Exception as e:
        issues.append((response.status_code, response.text, e))
        continue
    html_content = response.content

    # Use Beautiful Soup to parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find all headers and the text underneath them
    headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
    for header in tqdm(headers, leave=False):
        if isinstance(header, str):
            header_text = header
            next_elems = []
        else:
            header_text = header.get_text()
            next_elems = header.find_next_siblings()
        # Find the next element(s) after the header
        
        text = ''
        for elem in next_elems:
            # Stop when we reach another header
            if elem.name.startswith('h'):
                break
            else:
                text += elem.get_text()
        headers.append(header_text)
        texts.append(text)